In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils 
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('/content/drive/MyDrive/Winter/Public.csv', low_memory=False,encoding='utf-8') ### 데이터셋 불러오기
df = df.iloc[:, [4,6]] ### 분류번호와 제목만 가져옵니다.




In [3]:
df.drop_duplicates(subset = ['title'], inplace=True)  #제목에 겹치는게 많아 이럴 경우 중복 데이터를 삭제합니다.

df['title'] = df['title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "") #영어 분류는 제외하기로 하였기에 영어를 공백으로 제거

df['title'] = df['title'].str.replace('^ +', "")#공백과 띄어쓰기만 있는 셀들을 확인

df['class_no']= df['class_no'].str.replace("[^0-9]", "") #라벨지에 가끔 숫자 대신 글자가 들어가 있는 경우를 확인 이를 제거

df['class_no'] = df['class_no'].str.replace('^ +', "")#공백과 띄어쓰기만 있는 셀들을 확인

df.replace('', np.nan, inplace=True)#이를 drop하기 위하여 None으로 변경합니다.

df = df.dropna(how = 'any') #none은 전부 드랍

df = df.reset_index(drop = True) #이 경우 셀들의 배열이 맞지않아 배열을 다시 하여 줍니다.

X = df.iloc[:, 1] #제목만 따로 분리
y = df.iloc[:, 0] #라벨만 따로 분리
y = y.str[:1]  #라벨의 앞자리 즉 분류 자리수만 가져와 확인합니다.

y = y.astype(int)
y= y.iloc[:]
X= np.array(X)
y= np.array(y)
y = np_utils.to_categorical(y)
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state=1004)





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':


In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train) #X_train을 리스트의 형태로 변경
word_to_index = tokenizer.word_index
X_train_encoded = tokenizer.texts_to_sequences(X_train)
print(X_train_encoded[:5])





[[49053, 80020], [80021, 80022], [526, 1243], [220, 35521, 11308], [17052, 19533, 1190, 458, 1487]]


In [5]:
threshold = 2
total_cnt = len(word_to_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
vocab_size = len(word_to_index) + 1
print('제목의 최대 길이 : %d' % max(len(sample) for sample in X_train_encoded))
print('단어의 갯수 :' , total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합(vocabulary)에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

제목의 최대 길이 : 50
단어의 갯수 : 218947
등장 빈도가 1번 이하인 희귀 단어의 수: 138928
단어 집합(vocabulary)에서 희귀 단어의 비율: 63.452799079229216
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 13.247362027305615


In [6]:

X_train_padded = pad_sequences(X_train_encoded, maxlen = 50)
print(X_train_padded[:5])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
  49053 80020]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
  80021 80022]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
    526  1243]
 [    0     0     0     0     0     0     0     0     0     0     0

In [ ]:


hidden_units = 64
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(LSTM(hidden_units))
model.add(keras.layers.Dense(10, activation='softmax')) 
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
result = model.fit(X_train_padded, y_train, epochs=3, batch_size=64)
print("training complete")


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 16)          3503168   
                                                                 
 lstm_2 (LSTM)               (None, 64)                20736     
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 3,524,554
Trainable params: 3,524,554
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
 254/5451 [>.............................] - ETA: 8:53 - loss: 0.3083 - accuracy: 0.3636

In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = 40)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1]))